# Initialiazation Packages

In [1]:
println("Loading libraries")
using LinearAlgebra
using PyPlot
using PyCall
using LaTeXStrings
using QuantumToolbox
using DelimitedFiles, NLsolve
using Random, Distributions, Dates, Polynomials, StatsBase 
using JLD
println("Done")

Loading libraries
Done


In [2]:
println("Loading internal modules")
using Pkg
Pkg.activate("..")

include("../src/Open_2KPOs_QTB_functions.jl")
using .Coupled_KPOs_QTB_functions
println("Done")

Loading internal modules


  Activating project at `c:\Users\edson\Desktop\Research\Kerr_system\Coupled_KPOs`


Done


### Convergency: Minimum Fock basis for converged chaotic states

In [19]:
N1, N2 = 15, 10 
# Paramters
p = Δ1, K1, ξ11, ξ21, Δ2, K2, ξ12, ξ22, γ =  0., 1., 0., 5., 0., 1., 0., 5., 1.
Neff = 1.
κ1, κ2  =  0.025, 0.025
n_th1, n_th2 = 0.01, 0.01;

In [20]:
println("Defining H and dissipators for N1 and N2")
#Full closed Hamiltonian - No symmetries
N = N1
H_N1 = H_full(p, N);

a1, ad1, a2, ad2 = tensor(destroy(N), qeye(N)), tensor(create(N), qeye(N)), tensor(qeye(N), destroy(N)), tensor(qeye(N), create(N));
n1, n2 = tensor(num(N), qeye(N)), tensor(qeye(N), num(N));
#Defines the disipators for the master equation as $√Γ_i L_i$
c_ops_N1 = [sqrt(κ1 * (n_th1 + 1)) * a1, sqrt(κ1 * n_th1) * ad1, sqrt(κ2 * (n_th2 + 1)) * a2, sqrt(κ2 * n_th2) * ad2];


N = N2
#Full closed Hamiltonian - No symmetries
H_N2 = H_full(p, N2);

a1, ad1, a2, ad2 = tensor(destroy(N), qeye(N)), tensor(create(N), qeye(N)), tensor(qeye(N), destroy(N)), tensor(qeye(N), create(N));
n1, n2 = tensor(num(N), qeye(N)), tensor(qeye(N), num(N));
#Defines the disipators for the master equation as $√Γ_i L_i$
c_ops_N2 = [sqrt(κ1 * (n_th1 + 1)) * a1, sqrt(κ1 * n_th1) * ad1, sqrt(κ2 * (n_th2 + 1)) * a2, sqrt(κ2 * n_th2) * ad2];
println("Done")

Defining H and dissipators for N1 and N2
Done


In [21]:
println("Calculating Liouvillian's eigenvalues for Fock N1 = $(N1)")
t = time()
L_N1 = liouvillian(H_N1, c_ops_N1)
vals_N1, vecs_N1 = eigen(Matrix(L_N1)) #Convetional Diagonalization
sort(vals_N1, by=abs)
println("Time for Liouvillian Diagonalization: $(round((t-time())/60, digits = 3))min")

println("Calculating Liouvillian's eigenvalues for Fock N2 = $(N2)")
t = time()
L_N2 = liouvillian(H_N2, c_ops_N2)
vals_N2, vecs_N2 = eigen(Matrix(L_N2)) #Convetional Diagonalization
sort(vals_N2, by=abs)
println("Time for Liouvillian Diagonalization: $(round((t-time())/60, digits = 3))min")

Calculating Liouvillian's eigenvalues for Fock N1 = 15


LoadError: OutOfMemoryError()

In [ ]:
println("Calculating Energy difference")
se = abs.(E[1:length(E2)] - E2)
n_se = (findall(x -> x > 1e-3, se))[1] - 1
E_conv = n_se

fig = figure(figsize=(8,4), layout= "constrained")
gs = fig.add_gridspec(1,2)
element(i,j) = get(gs, (i,j)) # starts at 0 to N-1
slice(i,j) = pycall(pybuiltin("slice"), PyObject, i,j)

ax = fig.add_subplot(element(0,0))
plot(1:length(E_S_even2), (E_S_even[1:length(E_S_even2)] - E_S_even2), ".",label="S Even")
plot(1:length(E_S_odd2), (E_S_odd[1:length(E_S_odd2)] - E_S_odd2),".", label="S Odd")
plot(1:length(E_A_even2), (E_A_even[1:length(E_A_even2)] - E_A_even2),".", label="S Even")
plot(1:length(E_A_odd2), (E_A_odd[1:length(E_A_odd2)] - E_A_odd2), ".",label="AS Odd")
plot(1:length(E_S_even2), range(-1e-3,-1e-3,length=length(E_S_even2)), label=L"10^{-3}")
xlabel("state")
ylabel(L"E_{%$(N1)} - E_{%$(N2)}")
legend(fontsize=12)
title("#levels = $(sum(E_conv))",fontsize=12)


title("#levels = $(sum(state_conv))",fontsize=12)
fig.suptitle(L"Δ"*"= $(p[1]), "*L"ξ_{2}"*"= $(p[3]), γ = $(p[5]),",fontsize=14)

savefig("C:/Users/edson/Desktop/Research/Kerr_system/Coupled_kerr/codes/Quantum/Convergency/Convergency_p_$(p)_N_$(N2).png")
